In [1]:
!pip install git+https://github.com/seraphli/od-python.git

  Cloning https://github.com/seraphli/od-python.git to /tmp/pip-req-build-dnnjzwvm
  Running setup.py bdist_wheel for od-python ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-39nwjjy6/wheels/96/a0/9d/73ecc0fedd17a4cb6134556ad3c0386645657647205cc7062e
Successfully built od-python
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
import time
import od_python
from od_python.rest import ApiException
from pprint import pprint

In [3]:
premier_leagues = [
    "The International 2015", "The International 2017", "The International 2016", "The International 2013 Pass",
    "The International 2019", "The International 2014", "The International 2018", "The International 2012", 
    "EPICENTER Major 2019", "MDL Disneyland® Paris Major", "DreamLeague Season 11", 
    "World Electronic Sports Games 2018-2019", "The Chongqing Major", "The Kuala Lumpur Major",
    "CHINA DOTA2 SUPER MAJOR", "ESL One Birmingham 2018 powered by Intel", "MDL Changsha ",
    "EPICENTER XL", "Dota2 Asia Championships 2018", "World Electronic Sports Games 2018-2019",
    "ESL One Katowice 2018 powered by Intel", "DreamLeague season 8", "ESL One Hamburg 2017",
    "2017 Mars Dota 2 League", "NESO 4th National E-Sports ShenZhen Open Tournament- Galaxy Battles",
    "EPICENTER: Moscow", "Zotac Cup Masters", "StarLadder | i-League", "Kiev Major", "Dota 2 Asia Championships 2017",
    "Dota Pit League Season Five", "World Electronic Sports Games 2017", "ESL One Genting 2017",
    "World Cyber Arena 2016 Grand Finals", "Northern Arena BEAT Invitational presented by Bell",
    "The Summit 5", "Nanyang Championships Season2", "ESL One Frankfurt 2016", "The Manila Major 2016",
    "EPICENTER: Moscow Season 2", "WePlay League Season 3", "ESL One Manila 2016", "Dota Pit League Season Four",
    "2015 MarsTV Dota 2 League", "World Cyber Arena 2015 GRAND FINAL", "The Summit 4", "Nanyang Championships",
    "MLG World Finals 2015", "ESL One New York", "Dota Pit League Season Three", "Esportal Dota 2 League",
    "ESL One Frankfurt 2015", "ASUS ROG DreamLeague Season 3", "joinDOTA MLG Pro League Season 2", 
    "2015 MarsTV Dota 2 League", "SL i-League Invitational Season 3", "The Summit 3 ", "Dota 2 Champions League",
    "joinDOTA League", "Dota Pit League Season Two", "DotaCinema Presents the XMG Captains Draft 2.0",
    "The Summit 2", "ASUS ROG DreamLeague Season 2", "DotA 2 Champions League Season 4", "Excellent Moscow Cup Season 2",
    "ESL One New York", "ESL One Frankfurt 2014", "ASUS ROG DreamLeague Season 1", "The Summit", "Nexon Sponsorship League Season 3",
    "G-League 2013", "Nexon Sponsorship League Season 2 & Gama Brothers Courier", "RaidCall EMS One Fall 2013",
    "WePlay.TV Dota 2 League - Season 2", "2013 National Electronic Sports Tournament", "Nexon Sponsorship League",
    "Electronic Sports World Cup 2013", "The Defense Season 4", "Premier League Season 5 ", "RaidCall Dota 2 League Season 3",
    "REDBULL Esports Champion League 2013", "RaidCall EMS One Summer 2013", "Alienware Cup - 2013 Season 1",
    "Dota 2 Super League", "DreamHack ASUS ROG Dota 2 Tournament", "G-1 Champions League Season 5", "The Premier League Season 4",
    "RaidCall EMS One Spring 2013", "RaidCall Dota 2 League Season 2", "The Defense 3", "G-League 2013",
    "RaidCall Dota 2 League", "The Defense"    
]

international_league_ids = [2733, 5401, 4664, 65006, 10749, 600, 9870, 65001]

In [4]:
international_and_qualifiers = list()
all_pro_leagues = list()

# create an instance of the API class
league_api = od_python.LeaguesApi()

try:
    api_response = league_api.leagues_get()
    for entry in api_response:
        if entry.name in premier_leagues:
            international_and_qualifiers.append(entry.leagueid)
        if entry.tier == "premium" or entry.tier == "professional":
            all_pro_leagues.append(entry.leagueid)
except ApiException as e:
    print("Exception when calling LeaguesApi->leagues_get: %s\n" % e)


## Notes on Getting League IDs
* international_and_qualifiers contains all leagues that are "premier", or are most likely to mean a team qualifies for the international, or has a very large prize pool
* all_pro_leagues contains all leagues that the API deems either "professional" or "premium" these are likely the most common leagues that people watch or pay money to watch

In [5]:
def get_pro_matches(league_list):
    relevant_matches = list()
    pro_matches_api = od_python.ProMatchesApi()

    # Highest Match_ID as of 11/11/2019 - 5108977506
    match_id = 5108977506
    counter = 0

    while match_id > 17955123:
        try:
            if counter > 40:
                time.sleep(90)
                counter = 0
            else:
                api_response = pro_matches_api.pro_matches_get(less_than_match_id=match_id)
                counter = counter + 1
                for entry in api_response:
                    if entry.leagueid in league_list:
                        relevant_matches.append(entry.match_id)
                    if entry.match_id < match_id:
                        match_id = entry.match_id
        except ApiException as e:
            print("Exception when calling LeaguesApi->leagues_get: %s\n" % e)
            match_id = 0
            
    return relevant_matches


In [6]:
# Get Strictly matches that occur in International Tournaments
inter_match_ids = get_pro_matches(international_league_ids)

In [10]:
with open('international_matches.txt', 'a') as f:
    for item in inter_match_ids:
        f.write("%s\n" % item)

In [11]:
# Get matches that occur in International Tournaments and Premier Tournaments
time.sleep(90)
premier_match_ids = get_pro_matches(international_and_qualifiers)

In [12]:
with open('premier_matches.txt', 'a') as f:
    for item in premier_match_ids:
        f.write("%s\n" % item)

In [13]:
# Get all matches that the API considers professional or premium
time.sleep(90)
pro_match_ids = get_pro_matches(all_pro_leagues)

In [14]:
with open('pro_matches.txt', 'a') as f:
    for item in pro_match_ids:
        f.write("%s\n" % item)